## Tidy data and the Pandas module
This notebook accompanies Topic 7, which is about "tidying data," or cleaning up tabular data for analysis purposes. It also introduces one of the most important Python modules for data analysis: Pandas! (not the bear)

## Part 0: Getting the data
Before beginning, you'll need to download several files containing the data for the exercises below.

__Exercise 0__ (ungraded). Run the code cell below to download the data. (This code will check if each dataset has already been downloaded and, if so, will avoid re-downloading it.)

In [1]:
import requests
import os
import hashlib
import io

def download(file, url_suffix=None, checksum=None):
    if url_suffix is None:
        url_suffix = file
        
    if not os.path.exists(file):
        if os.path.exists('.voc'):
            url = 'https://cse6040.gatech.edu/datasets/{}'.format(url_suffix)
        else:
            url = 'https://github.com/cse6040/labs-fa17/raw/master/datasets/{}'.format(url_suffix)
        print("Downloading: {} ...".format(url))
        r = requests.get(url)
        with open(file, 'w', encoding=r.encoding) as f:
            f.write(r.text)
            
    if checksum is not None:
        with io.open(file, 'r', encoding='utf-8', errors='replace') as f:
            body = f.read()
            body_checksum = hashlib.md5(body.encode('utf-8')).hexdigest()
            assert body_checksum == checksum, \
                "Downloaded file '{}' has incorrect checksum: '{}' instead of '{}'".format(file, body_checksum, checksum)
    
    print("'{}' is ready!".format(file))
    
datasets = {'iris.csv': 'd1175c032e1042bec7f974c91e4a65ae',
            'table1.csv': '556ffe73363752488d6b41462f5ff3c9',
            'table2.csv': '16e04efbc7122e515f7a81a3361e6b87',
            'table3.csv': '531d13889f191d6c07c27c3c7ea035ff',
            'table4a.csv': '3c0bbecb40c6958df33a1f9aa5629a80',
            'table4b.csv': '8484bcdf07b50a7e0932099daa72a93d',
            'who.csv': '59fed6bbce66349bf00244b550a93544',
            'who2_soln.csv': 'f6d4875feea9d6fca82ae7f87f760f44',
            'who3_soln.csv': 'fba14f1e088d871e4407f5f737cfbc06'}

for filename, checksum in datasets.items():
    download(filename, url_suffix='tidy/{}'.format(filename), checksum=checksum)
    
print("\n(All data appears to be ready.)")

'iris.csv' is ready!
'table1.csv' is ready!
'table2.csv' is ready!
'table3.csv' is ready!
'table4a.csv' is ready!
'table4b.csv' is ready!
'who.csv' is ready!
'who2_soln.csv' is ready!
'who3_soln.csv' is ready!

(All data appears to be ready.)


## Part 1: Tidy data
The overall topic for this lab is what we'll refer to as representing data <i>relationally</i>. The topic of this part is a specific type of relational representation sometimes referred to as the <i>tidy</i> (as opposed to untidy or messy) form. The concept of tidy data was developed by <a href="http://hadley.nz/">Hadley Wickham</a>, a statistician and R programming maestro. Much of this lab is based on his tutorial materials (see below).<br><br>
If you know <a href="https://en.wikipedia.org/wiki/SQL">SQL</a>, then you are already familiar with relational data representations. However, we might discuss it a little differently from the way you may have encountered the subject previously. The main reason is our overall goal in the class: to build data <i>analysis</i> pipelines. If our end goal is analysis, then we often want to extract or prepare data in a way that makes analysis easier.<br><br>
You may find it helpful to also refer to the original materials on which this lab is based:
- Wickham's R tutorial on making data tidy: http://r4ds.had.co.nz/tidy-data.html
- The slides from a talk by Wickham on the concept: http://vita.had.co.nz/papers/tidy-data-pres.pdf
- Wickham's more theoretical paper of "tidy" vs. "untidy" data: http://www.jstatsoft.org/v59/i10/paper

## What is tidy data?
To build your intuition, consider the following data set collected from a survey or study.<br><br>
Representation 1. <a href="https://en.wikipedia.org/wiki/Contingency_table">Two-way contigency table</a>.<br>
Pregnant	Not pregnant<br>
Male	0	5<br>
Female	1	4<br><br>
Representation 2. Observation list or "data frame."<br>
Gender	Pregnant	Count<br>
Male	Yes	0<br>
Male	No	5<br>
Female	Yes	1<br>
Female	No	4<br><br>
These are two entirely equivalent ways of representing the same data. However, each may be suited to a particular task.<br><br>
For instance, Representation 1 is a typical input format for statistical routines that implement Pearson's $χ^{2}$-test, which can check for independence between factors. (Are gender and pregnancy status independent?) By contrast, Representation 2 might be better suited to regression. (Can you predict relative counts from gender and pregnancy status?)<br><br>
While <a href="http://simplystatistics.org/2016/02/17/non-tidy-data/">Representation 1 has its uses</a>, Wickham argues that Representation 2 is often the cleaner and more general way to supply data to a wide variety of statistical analysis and visualization tasks. He refers to Representation 2 as <i>tidy</i> and Representation 1 as <i>untidy</i> or <i>messy</i>.<br>

The term "messy" is, as Wickham states, not intended to be perjorative since "messy" representations may be exactly the right ones for particular analysis tasks, as noted above.
<br><br>
__Definition: Tidy datasets.__ More specifically, Wickham defines a tidy data set as one that can be organized into a 2-D table such that
1. each column represents a variable;
2. each row represents an observation;
3. each entry of the table represents a single value, which may come from either categorical (discrete) or continuous spaces.

Here is a visual schematic of this definition, taken from <a href="http://r4ds.had.co.nz/images/tidy-1.png">another source</a>:
<br><br>
This definition appeals to a statistician's intuitive idea of data he or she wishes to analyze. It is also consistent with tasks that seek to establish a functional relationship between some response (output) variable from one or more independent variables.
<br><br>
A computer scientist with a machine learning outlook might refer to columns as <i>features</i> and rows as <i>data points</i>, especially when all values are numerical (ordinal or continuous).<br><br>
__Definition: Tibbles.__ Here's one more bit of terminology: if a table is tidy, we will call it a <i>tidy table</i>, or <i>tibble</i>, for short.

## Part 2: Tidy Basics and Pandas
In Python, the <a href="http://pandas.pydata.org/">Pandas</a> module is a convenient way to store tibbles. If you know <a href="http://r-project.org/">R</a>, you will see that the design and API of Pandas's data frames derives from <a href="https://stat.ethz.ch/R-manual/R-devel/library/base/html/data.frame.html">R's data frames</a>.<br><br>
In this part of this notebook, let's look at how Pandas works and can help us store Tidy data.<br>

Consider the famous <a href="https://en.wikipedia.org/wiki/Iris_flower_data_set">Iris data set</a>. It consists of 50 samples from each of three species of Iris (<i>Iris setosa</i>, <i>Iris virginica</i>, and <i>Iris versicolor</i>). Four features were measured from each sample: the lengths and the widths of the <a href="https://en.wikipedia.org/wiki/Sepal">sepals</a> and <a href="https://en.wikipedia.org/wiki/Petal">petals</a>.<br><br>
The following code uses Pandas to read and represent this data in a Pandas data frame object, stored in a variable named `irises`.

In [2]:
# Some modules you'll need in this part
import pandas as pd
from io import StringIO
from IPython.display import display

# Ignore this line. It will be used later.
SAVE_APPLY = getattr(pd.DataFrame, 'apply')

irises = pd.read_csv('iris.csv')
print("=== Iris data set: {} rows x {} columns. ===".format(irises.shape[0], irises.shape[1]))
display (irises.head())

=== Iris data set: 150 rows x 5 columns. ===


,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In a Pandas data frame, every column has a name (stored as a string) and all values within the column must have the same primitive type. This fact makes columns different from, for instance, lists.

In addition, every row has a special column, called the data frame's <i>index</i>. (Try printing `irises.index`.) Any particular index value serves as a name for its row; these index values are usually integers but can be more complex types, like tuples.

In [3]:
print(irises.index)

RangeIndex(start=0, stop=150, step=1)


Separate from the index values (row names), you can also refer to rows by their integer offset from the top, where the first row has an offset of `0` and the last row has an offset of `n-1` if the data frame has `n` rows. You'll see that in action in Exercise 1, below.

__Exercise 1 (ungraded)__. Run the following commands to understand what each one does. If it's not obvious, try reading the <a href="http://pandas.pydata.org/">Pandas</a> documentation or going online to get more information.

In [4]:
irises.describe()
irises['sepal length'].head()
irises[["sepal length", "petal width"]].head()
irises.iloc[5:10]
irises[irises["sepal length"] > 5.0]
irises["sepal length"].max()
irises['species'].unique()
irises.sort_values(by="sepal length", ascending=False).head(1)
irises.sort_values(by="sepal length", ascending=False).iloc[5:10]
irises.sort_values(by="sepal length", ascending=False).loc[5:10]
irises['x'] = 3.14
irises.rename(columns={'species': 'type'})
del irises['x']

In [5]:
print("\n=== `irises.describe()`: Prints summary statistics ===\n\n{}".format(irises.describe()))
print("\n=== `irises['sepal length'].head()`: Dumps the first few rows of a given column ===\n\n{}".format(irises['sepal length'].head()))
print('\n=== `irises[["sepal length", "petal width"]].head()`: Dumps the first few rows of several specific columns ===\n\n{}'.format(irises[["sepal length", "petal width"]].head()))
print("\n=== `irises.iloc[5:10]`: Selects rows at a certain integer offset and range ===\n\n{}".format(irises.iloc[5:10]))
print('\n=== `irises[irises["sepal length"] > 5.0]`: Selects the subset of rows satisfying some condition (here, where sepal length is strictly more than 5) ===\n\n{}'.format(irises[irises["sepal length"] > 5.0]))
print('\n=== `irises["sepal length"].max()`: Returns the largest value of a given column ===\n\n{}'.format(irises["sepal length"].max()))
print("\n=== `irises['species'].unique()`: Returns a list of unique values in a given column ===\n\n{}".format(irises['species'].unique()))
print('\n=== `irises.sort_values(by="sepal length", ascending=False).head(1)`: Returns the observation with the longest sepal length ===\n\n{}'.format(irises.sort_values(by="sepal length", ascending=False).head(1)))
print('\n=== `irises.sort_values(by="sepal length", ascending=False).iloc[5:10]`: Returns the observations whose ranks, in highest sepal length, are 5-9 inclusive ===\n\n{}'.format(irises.sort_values(by="sepal length", ascending=False).iloc[5:10]))
print('\n=== `irises.sort_values(by="sepal length", ascending=False).loc[5:10]`: Returns the observations between the one whose row ID is 5 and the one that is 10, in highest sepal length, are 5-9 inclusive ===\n\n{}'.format(irises.sort_values(by="sepal length", ascending=False).loc[5:10]))

irises['x'] = 3.14
print("\n=== `irises['x'] = 3.14`: Creates a new column (variable) named `'x'` and sets its value to 3.14 ===\n\n{}".format(irises.head()))

irises2 = irises.rename(columns={'species': 'type'})
print("\n=== irises.rename(columns={{'species': 'type'}}): Change the name of a column (variable) ===\n\n{}".format(irises2))

del irises['x']
print("\n=== `del irises['x']`: Removes a column ===\n\n{}".format(irises.head()))


=== `irises.describe()`: Prints summary statistics ===

       sepal length  sepal width  petal length  petal width
count    150.000000   150.000000    150.000000   150.000000
mean       5.843333     3.057333      3.758000     1.199333
std        0.828066     0.435866      1.765298     0.762238
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.350000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.500000

=== `irises['sepal length'].head()`: Dumps the first few rows of a given column ===

0    5.1
1    4.9
2    4.7
3    4.6
4    5.0
Name: sepal length, dtype: float64

=== `irises[["sepal length", "petal width"]].head()`: Dumps the first few rows of several specific columns ===

   sepal length  petal width
0           5.1          0.2
1           4.9          0.2
2           4.7          0.2
3       

## Merging data frames: join operations
Another useful operation on data frames is <a href="http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html">merging</a>.

For instance, consider the following two tables, `A` and `B`:<br>
__country	year	cases__<br>
Afghanistan	1999	745<br>
Brazil	1999	37737<br>
China	1999	212258<br>
Afghanistan	2000	2666<br>
Brazil	2000	80488<br>
China	2000	213766<br>

__country	year	population__<br>
Afghanistan	1999	19987071<br>
Brazil	1999	172006362<br>
China	1999	1272915272<br>
Afghanistan	2000	20595360<br>
Brazil	2000	174504898<br>
China	2000	1280428583<br>

Suppose we wish to combine these into a single table, `C`:

__country	year	cases	population__<br>
Afghanistan	1999	745	19987071<br>
Brazil	1999	37737	172006362<br>
China	1999	212258	1272915272<br>
Afghanistan	2000	2666	20595360<br>
Brazil	2000	80488	174504898<br>
China	2000	213766	1280428583<br>

In Pandas, you can perform this merge using the <a href="http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html">.merge() function</a>:

`C = A.merge (B, on=['country', 'year'])`

In this call, the `on=` parameter specifies the list of column names to use to align or "match" the two tables, `A` and `B`. By default, `merge()` will only include rows from `A` and `B` where all keys match between the two tables.

The following code cell demonstrates this functionality.

In [6]:
A_csv = """country,year,cases
Afghanistan,1999,745
Brazil,1999,37737
China,1999,212258
Afghanistan,2000,2666
Brazil,2000,80488
China,2000,213766"""

with StringIO(A_csv) as fp:
    A = pd.read_csv(fp)
print("=== A ===")
display(A)

=== A ===


,country,year,cases
0,Afghanistan,1999,745
1,Brazil,1999,37737
2,China,1999,212258
3,Afghanistan,2000,2666
4,Brazil,2000,80488
5,China,2000,213766


In [7]:
B_csv = """country,year,population
Afghanistan,1999,19987071
Brazil,1999,172006362
China,1999,1272915272
Afghanistan,2000,20595360
Brazil,2000,174504898
China,2000,1280428583"""

with StringIO(B_csv) as fp:
    B = pd.read_csv(fp)
print("\n=== B ===")
display(B)


=== B ===


,country,year,population
0,Afghanistan,1999,19987071
1,Brazil,1999,172006362
2,China,1999,1272915272
3,Afghanistan,2000,20595360
4,Brazil,2000,174504898
5,China,2000,1280428583


In [8]:
C = A.merge(B, on=['country', 'year'])
print("\n=== C = merge(A, B) ===")
display(C)


=== C = merge(A, B) ===


,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Brazil,1999,37737,172006362
2,China,1999,212258,1272915272
3,Afghanistan,2000,2666,20595360
4,Brazil,2000,80488,174504898
5,China,2000,213766,1280428583


__Joins.__ This default behavior of keeping only rows that match both input frames is an example of what relational database systems call an inner-join operation. But there are several other types of joins.
- <i>Inner-join ($A$, $B$)</i> (default): Keep only rows of $A$ and $B$ where the on-keys match in both.
- <i>Outer-join ($A$, $B$)</i>: Keep all rows of both frames, but merge rows when the on-keys match. For non-matches, fill in missing values with not-a-number ($NaN$) values.
- <i>Left-join ($A$, $B$)</i>: Keep all rows of $A$. Only merge rows of $B$ whose on-keys match $A$.
- <i>Right-join ($A$, $B$)</i>: Keep all rows of $B$. Only merge rows of $A$ whose on-keys match $B$.

You can use `merge`'s `how=...` parameter, which takes the (string) values, '`inner`', '`outer`', '`left`', and '`right`'. Here is an example of an outer join.

## Apply functions to data frames
Another useful primitive is `apply()`, which can apply a function to a data frame or to a series (column of the data frame).

In [9]:
with StringIO("""x,y,z
bug,1,d
rug,2,d
lug,3,d
mug,4,d""") as fp:
    D = pd.read_csv(fp)
print("=== D ===")
display(D)

with StringIO("""x,y,w
hug,-1,e
smug,-2,e
rug,-3,e
tug,-4,e
bug,1,e""") as fp:
    E = pd.read_csv(fp)
print("\n=== E ===")
display(E)

print("\n=== Outer-join (D, E) ===")
display(D.merge(E, on=['x', 'y'], how='outer'))

print("\n=== Left-join (D, E) ===")
display(D.merge(E, on=['x', 'y'], how='left'))

print("\n=== Right-join (D, E) ===")
display(D.merge(E, on=['x', 'y'], how='right'))


print("\n=== Inner-join (D, E) ===")
display(D.merge(E, on=['x', 'y']))

=== D ===


,x,y,z
0,bug,1,d
1,rug,2,d
2,lug,3,d
3,mug,4,d



=== E ===


,x,y,w
0,hug,-1,e
1,smug,-2,e
2,rug,-3,e
3,tug,-4,e
4,bug,1,e



=== Outer-join (D, E) ===


...

,x,y,z,w
0,bug,1,d,e
1,rug,2,d,NaN
2,lug,3,d,NaN
3,mug,4,d,NaN



=== Right-join (D, E) ===


,x,y,z,w
0,bug,1,d,e
1,hug,-1,NaN,e
2,smug,-2,NaN,e
3,rug,-3,NaN,e
4,tug,-4,NaN,e



=== Inner-join (D, E) ===


,x,y,z,w
0,bug,1,d,e


In [10]:
display(C)

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Brazil,1999,37737,172006362
2,China,1999,212258,1272915272
3,Afghanistan,2000,2666,20595360
4,Brazil,2000,80488,174504898
5,China,2000,213766,1280428583


For instance, suppose we wish to convert the year into an abbrievated two-digit form. The following code will do it:

In [11]:
G = C.copy()
G['year'] = G['year'].apply(lambda x: "'{:02d}".format(x % 100))
display(G)

,country,year,cases,population
0,Afghanistan,'99,745,19987071
1,Brazil,'99,37737,172006362
2,China,'99,212258,1272915272
3,Afghanistan,'00,2666,20595360
4,Brazil,'00,80488,174504898
5,China,'00,213766,1280428583


__Exercise 2__ (2 points). Suppose you wish to compute the prevalence, which is the ratio of cases to the population.

The simplest way to do it is as follows:

`G['prevalence'] = G['cases'] / G['population']`

However, for this exercise, try to figure out how to use `apply()` to do it instead. To figure that out, you'll need to consult the documentation for `apply()` or go online to find some hints.

Implement your solution in a function, `calc_prevalence(G)`, which given `G` returns a new copy `H` that has a column named '`prevalence`' holding the correctly computed prevalence values.

Although there is the easy solution above, the purpose of this exercise is to force you to learn more about how `apply()` works, so that you can "apply" it in more settings in the future.

In [12]:
def calc_prevalence(G):
    assert 'cases' in G.columns and 'population' in G.columns
    def calc_ratio(observation):
        return observation['cases'] / observation['population']

    H = G.copy()
    H['prevalence'] = H.apply(calc_ratio, axis=1)
    return H

In [13]:
# Test cell: `prevalence_test`

H = calc_prevalence(G)
display(H) # Displayed `H` should have a 'prevalence' column

Easy_prevalence_method = G['cases'] / G['population']
assert (H['prevalence'] == Easy_prevalence_method).all(), "One or more prevalence values is incorrect."

print("Prevalance values seem correct. But did you use `apply()?` Let's see...")

# Tests that you actually used `apply()` in your function:
def apply_fail():
    raise ValueError("Did you really use apply?")
setattr(pd.DataFrame, 'apply', apply_fail)
try:
    calc_prevalence(G)
except (ValueError, TypeError):
    setattr(pd.DataFrame, 'apply', SAVE_APPLY)
    print("You used `apply()`. You may have even used it as intended.")
else:
    setattr(pd.DataFrame, 'apply', SAVE_APPLY)
    assert False, "Are you sure you used `apply()`?"
    

print("\n(Passed!)")

,country,year,cases,population,prevalence
0,Afghanistan,'99,745,19987071,0.000037
1,Brazil,'99,37737,172006362,0.000219
2,China,'99,212258,1272915272,0.000167
3,Afghanistan,'00,2666,20595360,0.000129
4,Brazil,'00,80488,174504898,0.000461
5,China,'00,213766,1280428583,0.000167


Prevalance values seem correct. But did you use `apply()?` Let's see...
You used `apply()`. You may have even used it as intended.

(Passed!)


## Part 3 : Tibbles and Bits
Now let's start creating and manipulating tibbles.

In [14]:
import pandas as pd  # The suggested idiom
from io import StringIO

from IPython.display import display # For pretty-printing data frames

__Exercise 3__ (3 points). Write a function, `canonicalize_tibble(X)`, that, given a tibble `X`, returns a new copy `Y` of `X` in <i>canonical order</i>. We say `Y` is in canonical order if it has the following properties.
1. The variables appear in sorted order by name, ascending from left to right.
2. The rows appear in lexicographically sorted order by variable, ascending from top to bottom.
3. The row labels (`Y.index`) go from 0 to `n-1`, where `n` is the number of observations.

For instance, here is a __non-canonical tibble__ ...<br>
c	a	b<br>
2	hat	x	1<br>
0	rat	y	4<br>
3	cat	x	2<br>
1	bat	x	2<br>
<br>
... and here is its __canonical counterpart.__<br>
a	b	c<br>
0	x	1	hat<br>
1	x	2	bat<br>
2	x	2	cat<br>
3	y	4	rat<br>
<br>
A partial solution appears below, which ensures that Property 1 above holds. Complete the solution to ensure Properties 2 and 3 hold. Feel free to consult the <a href="http://pandas.pydata.org/pandas-docs/stable/api.html">Pandas API</a>.
Hint. For Property 3, you may find `reset_index()` handy: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html

In [16]:
def canonicalize_tibble(X):
    # Enforce Property 1:
    var_names = sorted(X.columns)
    Y = X[var_names].copy()

    # Enforce Property 2:
    Y.sort_values(by=var_names, inplace=True)
    
    # Enforce Property 3:
    Y.reset_index(drop=True, inplace=True)

    return Y

In [17]:
# Test: `canonicalize_tibble_test`

# Test input
canonical_in_csv = """,c,a,b
2,hat,x,1
0,rat,y,4
3,cat,x,2
1,bat,x,2"""

with StringIO(canonical_in_csv) as fp:
    canonical_in = pd.read_csv(fp, index_col=0)
print("=== Input ===")
display(canonical_in)
print("")
    
# Test output solution
canonical_soln_csv = """,a,b,c
0,x,1,hat
1,x,2,bat
2,x,2,cat
3,y,4,rat"""

with StringIO(canonical_soln_csv) as fp:
    canonical_soln = pd.read_csv(fp, index_col=0)
print("=== True solution ===")
display(canonical_soln)
print("")

canonical_out = canonicalize_tibble(canonical_in)
print("=== Your computed solution ===")
display(canonical_out)
print("")

canonical_matches = (canonical_out == canonical_soln)
print("=== Matches? (Should be all True) ===")
display(canonical_matches)
assert canonical_matches.all().all()

print ("\n(Passed.)")

=== Input ===


,c,a,b
2,hat,x,1
0,rat,y,4
3,cat,x,2
1,bat,x,2



=== True solution ===


,a,b,c
0,x,1,hat
1,x,2,bat
2,x,2,cat
3,y,4,rat



=== Your computed solution ===


,a,b,c
0,x,1,hat
1,x,2,bat
2,x,2,cat
3,y,4,rat



=== Matches? (Should be all True) ===


,a,b,c
0,True,True,True
1,True,True,True
2,True,True,True
3,True,True,True



(Passed.)


__Exercise 4__ (1 point). Write a function, `tibbles_are_equivalent(A, B)` to determine if two tibbles, `A` and `B`, are equivalent. "Equivalent" means that `A` and `B` have identical variables and observations, up to permutations. If `A` and `B` are equivalent, then the function should return `True`. Otherwise, it should return `False`.

The last condition, "up to permutations," means that the variables and observations might not appear in the table in the same order. For example, the following two tibbles are equivalent:

__a	b	c__<br>
x	1	hat<br>
y	2	cat<br>
z	3	bat<br>
w	4	rat<br><br>
__b	c	a__<br>
2	cat	y<br>
3	bat	z<br>
1	hat	x<br>
4	rat	w<br>

By contrast, the following table would not be equivalent to either of the above tibbles.

__a	b	c__<br>
2	y	cat<br>
3	z	bat<br>
1	x	hat<br>
4	w	rat<br>

Note: Unlike Pandas data frames, tibbles conceptually do not have row labels. So you should ignore row labels.

In [18]:
def tibbles_are_equivalent(A, B):
    """Given two tidy tables ('tibbles'), returns True iff they are
    equivalent.
    """
    A_hat = canonicalize_tibble(A)
    B_hat = canonicalize_tibble(B)
    equal = (A_hat == B_hat)
    return equal.all().all()

In [19]:
# Test: `tibble_are_equivalent_test`

A = pd.DataFrame(columns=['a', 'b', 'c'],
                 data=list(zip (['x', 'y', 'z', 'w'],
                                [1, 2, 3, 4],
                                ['hat', 'cat', 'bat', 'rat'])))
print("=== Tibble A ===")
display(A)

# Permute rows and columns, preserving equivalence
import random

obs_ind_orig = list(range(A.shape[0]))
var_names = list(A.columns)

obs_ind = obs_ind_orig.copy()
while obs_ind == obs_ind_orig:
    random.shuffle(obs_ind)
    
while var_names == list(A.columns):
    random.shuffle(var_names)

B = A[var_names].copy()
B = B.iloc[obs_ind]

print ("=== Tibble B == A ===")
display(B)

print ("=== Tibble C != A ===")
C = A.copy()
C.columns = var_names
display(C)

assert tibbles_are_equivalent(A, B)
assert not tibbles_are_equivalent(A, C)
assert not tibbles_are_equivalent(B, C)

print ("\n(Passed.)")

=== Tibble A ===


,a,b,c
0,x,1,hat
1,y,2,cat
2,z,3,bat
3,w,4,rat


=== Tibble B == A ===


,c,b,a
2,bat,3,z
1,cat,2,y
3,rat,4,w
0,hat,1,x


=== Tibble C != A ===


,c,b,a
0,x,1,hat
1,y,2,cat
2,z,3,bat
3,w,4,rat



(Passed.)


## Basic tidying transformations: Melting and casting
Given a data set and a target set of variables, there are at least two common issues that require tidying.

### Melting
First, values often appear as columns. Table 4a is an example. To tidy up, you want to turn columns into rows:

Because this operation takes columns into rows, making a "fat" table more tall and skinny, it is sometimes called <i>melting</i>.

To melt the table, you need to do the following.

1. Extract the <i>column values</i> into a new variable. In this case, columns "`1999`" and "`2000`" of `table4` need to become the values of the variable, "`year`".
2. Convert the values associated with the column values into a new variable as well. In this case, the values formerly in columns "`1999`" and "`2000`" become the values of the "`cases`" variable.

In the context of a melt, let's also refer to "`year`" as the new <i>key</i> variable and "`cases`" as the new <i>value</i> variable.

__Exercise 5__ (4 points). Implement the melt operation as a function,

In [20]:
def melt(df, col_vals, key, value):
        ...

It should take the following arguments:
- `df`: the input data frame, e.g., `table4` in the example above;
- `col_vals`: a list of the column names that will serve as values;
- `key`: name of the new variable, e.g., `year` in the example above;
- `value`: name of the column to hold the values.

You may need to refer to the Pandas documentation to figure out how to create and manipulate tables. The bits related to <a href="http://pandas.pydata.org/pandas-docs/stable/indexing.html">indexing</a> and <a href="http://pandas.pydata.org/pandas-docs/stable/merging.html">merging</a> may be especially helpful.

In [21]:
def melt(df, col_vals, key, value):
    assert type(df) is pd.DataFrame
    keep_vars = df.columns.difference(col_vals)
    melted_sections = []
    for c in col_vals:
        melted_c = df[keep_vars].copy()
        melted_c[key] = c
        melted_c[value] = df[c]
        melted_sections.append(melted_c)
    melted = pd.concat(melted_sections)
    return melted

In [22]:
# Test: `melt_test`

table4a = pd.read_csv('table4a.csv')
print("\n=== table4a ===")
display(table4a)

m_4a = melt(table4a, col_vals=['1999', '2000'], key='year', value='cases')
print("=== melt(table4a) ===")
display(m_4a)

table4b = pd.read_csv('table4b.csv')
print("\n=== table4b ===")
display(table4b)

m_4b = melt(table4b, col_vals=['1999', '2000'], key='year', value='population')
print("=== melt(table4b) ===")
display(m_4b)

m_4 = pd.merge(m_4a, m_4b, on=['country', 'year'])
print ("\n=== inner-join(melt(table4a), melt (table4b)) ===")
display(m_4)

m_4['year'] = m_4['year'].apply (int)

table1 = pd.read_csv('table1.csv')
print ("=== table1 (target solution) ===")
display(table1)
assert tibbles_are_equivalent(table1, m_4)
print ("\n(Passed.)")


=== table4a ===


,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


=== melt(table4a) ===


,country,year,cases
0,Afghanistan,1999,745
1,Brazil,1999,37737
2,China,1999,212258
0,Afghanistan,2000,2666
1,Brazil,2000,80488
2,China,2000,213766



=== table4b ===


...


=== inner-join(melt(table4a), melt (table4b)) ===


,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Brazil,1999,37737,172006362
2,China,1999,212258,1272915272
3,Afghanistan,2000,2666,20595360
4,Brazil,2000,80488,174504898
5,China,2000,213766,1280428583


=== table1 (target solution) ===


,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583



(Passed.)


## Casting
The second most common issue is that an observation might be split across multiple rows. Table 2 is an example. To tidy up, you want to merge rows:

Because this operation is the moral opposite of melting, and "rebuilds" observations from parts, it is sometimes called <i>casting</i>.

Melting and casting are Wickham's terms from <a href="http://www.jstatsoft.org/v59/i10/paper">his original paper on tidying data</a>. In his more recent writing, <a href="http://r4ds.had.co.nz/tidy-data.html">on which this tutorial is based</a>, he refers to the same operation as <i>gathering</i>. Again, this term comes from Wickham's original paper, whereas his more recent summaries use the term <i>spreading</i>.

The signature of a cast is similar to that of melt. However, you only need to know the `key`, which is column of the input table containing new variable names, and the `value`, which is the column containing corresponding values.

__Exercise 6__ (4 points). Implement a function to cast a data frame into a tibble, given a key column containing new variable names and a value column containing the corresponding cells.

We've given you a partial solution that
- verifies that the given `key` and `value` columns are actual columns of the input data frame;
- computes the list of columns, `fixed_vars`, that should remain unchanged; and
- initializes and empty tibble.

Observe that we are asking your `cast()` to accept an optional parameter, `join_how`, that may take the values '`outer`' or '`inner`' (with '`outer`' as the default). Why do you need such a parameter?

In [23]:
def cast(df, key, value, join_how='outer'):
    """Casts the input data frame into a tibble,
    given the key column and value column.
    """
    assert type(df) is pd.DataFrame
    assert key in df.columns and value in df.columns
    assert join_how in ['outer', 'inner']
    
    fixed_vars = df.columns.difference([key, value])
    tibble = pd.DataFrame(columns=fixed_vars) # empty frame
    
    new_vars = df[key].unique()
    for v in new_vars:
        df_v = df[df[key] == v]
        del df_v[key]
        df_v = df_v.rename(columns={value: v})
        tibble = tibble.merge(df_v,
                              on=list(fixed_vars),
                              how=join_how)

    return tibble

In [24]:
# Test: `cast_test`

table2 = pd.read_csv('table2.csv')
print('=== table2 ===')
display(table2)

print('\n=== tibble2 = cast (table2, "type", "count") ===')
tibble2 = cast(table2, 'type', 'count')
display(tibble2)

assert tibbles_are_equivalent(table1, tibble2)
print('\n(Passed.)')

=== table2 ===


,country,year,type,count
0,Afghanistan,1999,cases,745
1,Afghanistan,1999,population,19987071
2,Afghanistan,2000,cases,2666
3,Afghanistan,2000,population,20595360
4,Brazil,1999,cases,37737
5,Brazil,1999,population,172006362
6,Brazil,2000,cases,80488
7,Brazil,2000,population,174504898
8,China,1999,cases,212258
9,China,1999,population,1272915272



=== tibble2 = cast (table2, "type", "count") ===


,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583



(Passed.)


## Separating variables
Consider the following table.

In [26]:
table3 = pd.read_csv('table3.csv')
display(table3)

,country,year,rate
0,Afghanistan,1999,745/19987071
1,Afghanistan,2000,2666/20595360
2,Brazil,1999,37737/172006362
3,Brazil,2000,80488/174504898
4,China,1999,212258/1272915272
5,China,2000,213766/1280428583


In this table, the `rate` variable combines what had previously been the `cases` and `population` data. This example is an instance in which we might want to <i>separate</i> a column into two variables.

In [15]:
from IPython.display import HTML
HTML(filename='7-main.html')